# 02. Train Model

ToDo:
    * Ensemble using majority voting
    * Fix metrics
    * Test xgboost
    * Test reciprocal
    * Create new feature (999)
    * Add log reg (improves a little bit)

Changelog:
    * Duplicando dados positivos para melhorar o balanceamento
    * Stack with LR

In [543]:
VERSION = '180612_v2'

In [544]:
# all imports
import sys
sys.path.insert(0, '../../src/')

import pandas as pd
import os
import transformers
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from transformers import (ModelTransformer, DataFrameColumnExtractor, ToDictTransformer)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn import cross_validation as cv
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.preprocessing import PolynomialFeatures

#evaluators
#gridcv

### Read data

In [545]:
DATA_DIR = '../../data/raw/'
TRAIN_DATASET_PATH = os.path.join(DATA_DIR, 'train.csv')
TEST_DATASET_PATH = os.path.join(DATA_DIR, 'test.csv')

In [546]:
df_train = pd.read_csv(TRAIN_DATASET_PATH, encoding='utf-8')
df_test = pd.read_csv(TEST_DATASET_PATH, encoding='utf-8')

### Train/Test Data

In [547]:
X = df_train.drop(['y', 'id'], axis=1)
y = df_train['y']#.astype(bool)

In [548]:
X_train, X_test, y_train, y_test = cv.train_test_split(X,
                                                       y,
                                                       test_size=0.25,
                                                       random_state=42)

In [549]:
X_train_yes = X_train[y_train == 1]
y_train_yes = y_train[y_train == 1]

X_train = pd.concat([X_train, X_train_yes])
y_train = pd.concat([y_train, y_train_yes])

### Train model

In [550]:
CATEGORICAL_FEATURES = [
    'job',
    'marital',
    'education',
    'default',
    'housing',
    'loan',
    'contact',
    'poutcome',
    'month',
    'day_of_week'
]

NUMERIC_FEATURES = [                    
    'age',
    'pdays',
    'emp.var.rate',
    'cons.price.idx',
    'cons.conf.idx',
    'euribor3m',
    'campaign',
    'previous',
    'nr.employed'
]

TO_APPLY_LOG = [
]

TO_APPLY_CUBE_ROOT = [

]

TO_APPLY_RECIPROCAL = [

]

TO_BIN = [

]

TRAINING_FEATURES = NUMERIC_FEATURES + CATEGORICAL_FEATURES
ALL_COLUMNS = TRAINING_FEATURES + ['id', 'y']

In [551]:
transformer_pipeline = make_pipeline(
    make_union(
        make_pipeline(
            make_union(
                make_pipeline(
                    DataFrameColumnExtractor(NUMERIC_FEATURES),
#                     Imputer(strategy="median", axis=0),
                ),
#                 make_pipeline(
#                     DataFrameColumnExtractor(TO_APPLY_RECIPROCAL),
# #                     Imputer(strategy="median", axis=0),
#                     FunctionTransformer(transformers.sum_1),
#                     FunctionTransformer(transformers.apply_reciprocal),
#                 )
            ),
            StandardScaler(),
#             PolynomialFeatures()
        ),
        make_pipeline(
            DataFrameColumnExtractor(CATEGORICAL_FEATURES),
            ToDictTransformer(),
            DictVectorizer(sparse=False)
        )
    )
)

In [552]:
predictor_pipeline = make_pipeline(
     make_union(
         ModelTransformer(LogisticRegression(random_state=42)),
         FunctionTransformer(transformers.all_columns)
     ),
     GradientBoostingClassifier(random_state=42, max_depth=4) 
)

#      make_union(
#          ModelTransformer(SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#             decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
#             max_iter=-1, probability=False, random_state=None, shrinking=True,
#             tol=0.001, verbose=False)),
#          FunctionTransformer(transformers.all_columns)
#      ),

In [553]:
pipeline= Pipeline([
    ('features', transformer_pipeline),
    ('predictor', predictor_pipeline)
])

In [554]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features', Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline', Pipeline(steps=[('dataframecolumnextractor', DataFrameColumnExtractor(columns=['age', 'pda...100, presort='auto', random_state=42,
              subsample=1.0, verbose=0, warm_start=False))]))])

### Evaluate model

In [555]:
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)

In [556]:
print(metrics.classification_report(y_test, y_pred))

print("Precision: "+ str(metrics.precision_score(y_test, y_pred)))
print("Recall: "+ str(metrics.recall_score(y_test, y_pred)))
print("F1 Score: "+ str(metrics.f1_score(y_test, y_pred, average='weighted')))

             precision    recall  f1-score   support

          0       0.93      0.95      0.94      8077
          1       0.51      0.43      0.47       985

avg / total       0.89      0.89      0.89      9062

Precision: 0.5053507728894173
Recall: 0.43147208121827413
F1 Score: 0.8885265478255482


### Generate submission file

In [565]:
DATA_DIR = '../../data/raw/'
TRAIN_DATASET_PATH = os.path.join(DATA_DIR, 'train.csv')
TEST_DATASET_PATH = os.path.join(DATA_DIR, 'test.csv')

In [566]:
df_train = pd.read_csv(TRAIN_DATASET_PATH, encoding='utf-8')
df_test = pd.read_csv(TEST_DATASET_PATH, encoding='utf-8')

In [567]:
X_train = df_train.drop(['y', 'id'], axis=1)
y_train = df_train['y']#.astype(bool)

X_test = df_test.drop(['id'], axis=1)

In [568]:
X_train_yes = X_train[y_train == 1]
y_train_yes = y_train[y_train == 1]

X_train = pd.concat([X_train, X_train_yes])
y_train = pd.concat([y_train, y_train_yes])

In [569]:
pipeline.fit(X, y)

Pipeline(steps=[('features', Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline', Pipeline(steps=[('dataframecolumnextractor', DataFrameColumnExtractor(columns=['age', 'pda...100, presort='auto', random_state=42,
              subsample=1.0, verbose=0, warm_start=False))]))])

In [570]:
pred_submission = pipeline.predict(X_test)

In [571]:
ids_submission = df_test['id']

In [572]:
# Save submission file
SUBMISSION_DATA_DIR = '../../data/submission/'
SUBMISSION_FILE_PATH = os.path.join(SUBMISSION_DATA_DIR, VERSION+'.csv')

df_submission = pd.DataFrame({'id':ids_submission, 'y':pred_submission})
df_submission.to_csv(SUBMISSION_FILE_PATH, index=False)

In [573]:
df_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9062 entries, 0 to 9061
Data columns (total 2 columns):
id    9062 non-null int64
y     9062 non-null int64
dtypes: int64(2)
memory usage: 141.7 KB
